# ModOAP - Téléchargement des blocs de texte de documents Gallica

Ce carnet propose de spécifier l'**identifiant ark** d'un document ou une **liste d'identifiants** dans une colonne d'un fichier excel, pour **télécharger les blocs de texte** dans un fichier structuré au format **json**.


In [ ]:
 #@markdown ### Préparation et connexion à un compte Google Drive
#@markdown Lancer cette cellule, puis cliquer sur le lien généré par Google pour connecter un compte Drive si demandé.

from google.colab import drive
import os

# chargement d'un google drive
if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')

import requests
from openpyxl import load_workbook
import urllib.request, urllib.error, urllib.parse
from urllib.error import HTTPError, URLError
import json
from bs4 import BeautifulSoup
try :
  import xmltodict
except :
  !pip -q install xmltodict
  import xmltodict
import unicodedata
import re 
import glob
from tqdm import tqdm




def remove_accents(s):
  # In : chaine avec caractères diachrités
  # Out : chaine sans accent
  return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def normalisation_titre(titre):
  # In : chaine titre
  # Out : chaine titre pour nom de fichier
  titre = remove_accents(titre)
  titre = re.sub('[^a-zA-Z0-9- ]', '', titre)
  titre = re.sub('[ ]', '_', titre)
  return "_".join(titre.split("_")[:6])

def get_var_ts(ts) :
  # In : text string (mot) (ordered dict key, list ou dict)
  # ajoute le contenu du mot au contenu du bloc texte
  if isinstance(ts, list) :
    for mot in ts :
      contenu = mot["@CONTENT"]
      text_block_words.append(contenu)
  else :
    contenu = ts["@CONTENT"]
    text_block_words.append(contenu)

def get_content(tlentry) :
  # In : textline (ordered dict key, list ou dict)
  # lance get_var_ts
  if isinstance(tlentry, list) :
    for tl in tlentry :
      get_var_ts(tl["String"])
  else : get_var_ts(tlentry["String"])

def get_var_tb(tb) :
  # In : textblock (ordered dict key, dict)
  # récupère la position et l'id du bloc texte
  # lance get_var_tb
  width = tb["@WIDTH"]
  height = tb["@HEIGHT"]
  hpos = tb["@HPOS"]
  vpos = tb["@VPOS"]
  id = tb["@ID"]
  get_content(tb["TextLine"])
  return width, height, hpos, vpos, id

def get_tb(tb_entry) :
  # In : textblock (ordered dict key, list ou dict)
  # récupère la position et l'id du bloc texte
  # lance get_var_tb
  if isinstance(tb_entry, list) :
    for tb in tb_entry :
      width, height, hpos, vpos, id = get_var_tb(tb)
  else : 
    width, height, hpos, vpos, id = get_var_tb(tb_entry)
  return width, height, hpos, vpos, id

def block_treatment(tb, num_page) :
  # In :  textblock (ordered dict key), numéro de page (int)
  # lance get_tb
  # ajoute la position et le contenu du bloc texte dans un dictionnaire
  global text_block_words
  text_block_words = []
  width, height, hpos, vpos, id = get_tb(tb)
  contenu = " ".join(text_block_words)
  position_dic = {"width" : width, "height" : height, "hpos" : hpos, "vpos" : vpos}
  dico_id = {"Page_Num" : num_page, "Position" : position_dic, "Content" : contenu}
  text_blocks[id] = dico_id

def infos_doc(ark):
  # In :  identifiant ark
  # Out : titre, date de publication du document (str)
  url_biblio = "https://gallica.bnf.fr/services/OAIRecord?ark="+ark
  s = requests.get(url_biblio, stream=True)
  bibliodico = xmltodict.parse(s.text)
  try :
    titre = bibliodico["results"]["title"]
  except :
    titre = "unknown"
  try :
    date_pub = bibliodico["results"]["date"]["#text"]
  except :
    date_pub = "unknown"
  return titre, date_pub


def nombre_pages(ark):
  # In :  identifiant ark
  # Out : nombre de pages (int)
  PAGINATION_BASEURL = 'https://gallica.bnf.fr/services/Pagination?ark='
  url = "".join([PAGINATION_BASEURL, ark])
  s = requests.get(url, stream=True)
  paginationdic = xmltodict.parse(s.text)
  nb_pages = int(paginationdic["livre"]["structure"]["nbVueImages"])
  return nb_pages

def ark_processing(ark) :
  # In :  identifiant ark
  # Out : fichier json contenant : 
    # titre, date, nombre de pages du document
    # id, position, contenu des blocs de textes de chaque page

  titre, date_pub = infos_doc(ark)
  titre_fichier = normalisation_titre(titre)  
  try :
    nombre_pagess = nombre_pages(ark)
  except :
    print("Pagination indisponible, document non-téléchargé : ", ark)

  print("Titre du document : ", titre)
  print("Dossier de destination : ", destination_dir)
  print("Téléchargement des blocs de texte :")

  info_doc = {"Titre" : titre, "Publication_Date" : date_pub, "Total_Pages" : nombre_pagess }
  global text_blocks 
  text_blocks = {}

  # Pour chaque page du document :
  for num_page in tqdm(range(1,nombre_pagess+1)) :
  #for num_page in range(1,10) : 
    #print("page ",num_page," / ",str(nombre_pagess))
    # Transforme le fichier OCR ALTO de la page en un dictionnaire :
    alto_url = 'https://gallica.bnf.fr/RequestDigitalElement?O='+ark+'&E=ALTO&Deb='+str(num_page)
    s = requests.get(alto_url, stream=True)
    altodic = xmltodict.parse(s.text)
    # Si un ou plusieurs blocs de texte sont directement présents dans le PrintSpace : 
    try :
      tb_entry = altodic['alto']["Layout"]["Page"]["PrintSpace"]["TextBlock"]
      if isinstance(tb_entry, list) :
        for tb in tb_entry :
          block_treatment(tb, num_page)
        
      else :
        block_treatment(tb_entry, num_page)
    except :
      pass
    # Si un ou plusieurs blocs composés sont présents dans le PrintSpace : 
    try :
      cb_entry = altodic['alto']["Layout"]["Page"]["PrintSpace"]["ComposedBlock"]
      if isinstance(cb_entry, list) :
        for cb in cb_entry :
          tb_entry = cb["TextBlock"]
          if isinstance(tb_entry, list) :
            for tb in tb_entry :
              block_treatment(tb, num_page)
          else :
            block_treatment(tb_entry, num_page)
      else :
        tb_entry = cb_entry["TextBlock"]
        if isinstance(tb_entry, list) :
          for tb in tb_entry :
            block_treatment(tb, num_page)
        else :
          block_treatment(tb_entry, num_page)
    except :
      pass
  # Création du fichier final :
  jsondic = {"Infos_Doc" : info_doc, "Text_Blocks" : text_blocks} 
  with open(os.path.join(destination_dir,str(date_pub)+"_"+titre_fichier+"_"+ark+"_texte_structure.json"), 'w') as jout :
    json.dump(jsondic, jout)
  print("Le texte du document a été sauvegardé dans le fichier ", os.path.join(destination_dir,str(date_pub)+"_"+titre_fichier+"_"+ark+"_texte_structure.json"))

def bnf2gall(arkbnf):
  # In :  identifiant ark au format type cb453908509
  # Out : identifiant ark au format type bpt6k9799524x (consultable sur Gallica)
  url = "https://catalogue.bnf.fr/ark:/12148/"+str(arkbnf)
  s = requests.get(url, stream=True)
  html = BeautifulSoup(s.content) # html5lib
  for link in html.findAll('a', {'class': 'exemplaire-action-visualiser'}):
    ark = link['href'].split("/")[-1]
  return ark

Mounted at /content/drive/


In [ ]:
#@markdown ### A partir d'un identifiant ARK :
#@markdown ---
#@markdown Entrer l'identifiant ARK d'un document, puis lancer la cellule.

ark = "btv1b8618381s" #@param {type:"string"}
#@markdown Exemple d'identifiant:
#@markdown bpt6k9799524x ou cb453908509

#@markdown ---
#@markdown Entrer le chemin du répertoire souhaité, puis lancer la cellule.

destination_dir = "/content/drive/MyDrive/dossier_test" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/datasets/

#@markdown Possibilité de copier/coller le chemin depuis la fenêtre de gauche : onglet "Fichiers" -> clic droit sur un dossier -> "Copier le chemin"

if ark.startswith("cb") :
  ark = bnf2gall(ark)
ark_processing(ark)


Titre du document :  Les Minutes de sable : mémorial / Alfred jarry
Dossier de destination :  /content/drive/MyDrive/dossier_test
Téléchargement des blocs de texte :


100%|██████████| 249/249 [01:40<00:00,  2.47it/s]

Le texte du document a été sauvegardé dans le fichier  /content/drive/MyDrive/dossier_test/1894_Les_Minutes_de_sable__memorial_btv1b8618381s_texte_structure.json


In [ ]:
#@markdown ### Ou bien à partir d'un fichier excel :
#@markdown ---

#@markdown Entrer le chemin vers le fichier (xlsx ou xlsm) et l'indice de la colonne, puis lancer la cellule.
chemin_fichier_xls = "/content/drive/MyDrive/dossier_test/FPLAB_3docs_pour_telechargement_strucure.xlsx" #@param {type:"string"}

#@markdown Exemple de chemin : /content/drive/MyDrive/Document/documents.xlsx

#@markdown ---
#@markdown Entrer l'indice de la colonne contenant les liens ARK :
colonne_ark = "A" #@param {type:"string"}
#@markdown Exemple d'indice : A

#@markdown ---
#@markdown Entrer le chemin du répertoire souhaité, puis lancer la cellule.

destination_dir = "/content/drive/MyDrive/dossier_test" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/datasets/

arks_done = [rk.split("_")[-3] for rk in glob.glob(os.path.join(destination_dir,"*.json"))]

try :
  if not os.path.exists(destination_dir):
      os.makedirs(destination_dir)
except :
  print("Le chemin de destination est incorrect")

arks_doc = []

# Chargement du fichier xls
try :
  classeur= load_workbook(chemin_fichier_xls)
except :
  print("Le fichier xls n'a pas été chargé correctement")

for onglet in classeur.sheetnames:
  onglet_courant = classeur[onglet]
  colonne = onglet_courant[colonne_ark]
  for cellule in colonne :
    if str(cellule.value).startswith("http") or str(cellule.value).startswith("ark"):
      arks_doc.append(str(cellule.value).split("/")[-1].strip())
    elif str(cellule.value).startswith("cb") or str(cellule.value).startswith("bp"):
      arks_doc.append(str(cellule.value))

print("{0} liens récupérés dans {1} onglets".format(len(arks_doc), len(classeur.sheetnames)))

arks_doc = set(arks_doc)

for ark in arks_doc :
  if ark.startswith("cb") :
    try :
      ark = bnf2gall(ark)
    except :
      pass
  if ark not in arks_done :
    try :
      ark_processing(ark)
    except :
      print("Le document {} n'a pas été téléchargé".format(ark))
      pass
  else : print("Le document {} est déjà téléchargé".format(ark))

3 liens récupérés dans 1 onglets
Titre du document :  Manuel d'histoire de France : contenant les tableaux d'histoire des mêmes auteurs / par MM. Meissas et Michelot
Dossier de destination :  /content/drive/MyDrive/dossier_test
Téléchargement des blocs de texte :


 67%|██████▋   | 113/168 [00:47<00:23,  2.35it/s]


Le document bpt6k96919287 n'a pas été téléchargé
Titre du document :  A. B. C. de l'histoire universelle : contenant les faits caractéristiques et les évènements principaux de chaque siècle depuis la création jusqu'à nos jours, et pouvant servir d'introduction et de récapitulation à l'étude de l'histoire (Nouvelle édition, corrigée et augmentée) / par Édouard W. d'Halluvin,...
Dossier de destination :  /content/drive/MyDrive/dossier_test
Téléchargement des blocs de texte :


100%|██████████| 200/200 [01:27<00:00,  2.28it/s]


Le texte du document a été sauvegardé dans le fichier  /content/drive/MyDrive/dossier_test/1867_A_B_C_de_lhistoire_universelle_bpt6k9690982q_texte_structure.json
Titre du document :  Histoire de France à l'usage des écoles primaires : cours élémentaire (5e édition) / par Gustave Hubault
Dossier de destination :  /content/drive/MyDrive/dossier_test
Téléchargement des blocs de texte :


100%|██████████| 116/116 [00:50<00:00,  2.30it/s]

Le texte du document a été sauvegardé dans le fichier  /content/drive/MyDrive/dossier_test/1880_Histoire_de_France_a_lusage_des_bpt6k96911895_texte_structure.json
